In [1]:
import requests
from bs4 import BeautifulSoup
import spacy
import json
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

C:\Users\fercy\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
link='https://www.bbc.co.uk/news/world-us-canada-65394202'
soup = BeautifulSoup(requests.get(link).content, "html.parser")

parr=str(soup.findAll("p")[0]).split(">")[0]
parr = parr[9:]
parr

'"ssrcss-1q0x1qg-Paragraph eq5iqo00"'

In [4]:
text=[]
for c in soup.findAll("p",attrs={"class": "ssrcss-1q0x1qg-Paragraph eq5iqo00"}): # this is for bbc content
    text.append(c.getText())
text=str(text)
print(text)

["British singer Ed Sheeran has appeared in a New York City court to deny that his song Thinking Out Loud copied Marvin Gaye's song Let's Get it On. ", "Heirs of Gaye's co-writer, Ed Townsend, argue that Sheeran, Warner Music Group and Sony Music Publishing owe them money for allegedly stealing the song.", 'As the case opened, a lawyer called Sheeran\'s use of Gaye\'s lyrics at his concerts a "smoking gun". ', "The Grammy winner, 32, testified that he did not copy from Gaye's 1973 hit.", 'Asked by lawyer Keisha Rice about another song he wrote, Take it Back, which contains the lyrics "plagiarism is hidden", Sheeran confirmed that he had written the words. ', '"Those are my lyrics, yep," he said, adding: "Can I give some context to them?"', "She said that if she wanted any context she would ask for it, and went on to ask him about concert footage recorded in Zurich showing him mixing lyrics from Gaye's song with Thinking Out Loud. ", 'Earlier, another lawyer for the family - civil right

In [5]:
per=0.2
nlp = spacy.load('en_core_web_trf')
doc= nlp(text)
tokens=[token.text for token in doc]
word_frequencies={}
for word in doc:
    if word.text.lower() not in list(STOP_WORDS):
        if word.text.lower() not in punctuation:
            if word.text not in word_frequencies.keys():
                word_frequencies[word.text] = 1
            else:
                word_frequencies[word.text] += 1
max_frequency=max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency
sentence_tokens= [sent for sent in doc.sents]
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():                            
                sentence_scores[sent]=word_frequencies[word.text.lower()]
            else:
                sentence_scores[sent]+=word_frequencies[word.text.lower()]
select_length=int(len(sentence_tokens)*per)
summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
final_summary=[word.text for word in summary]
summary=''.join(final_summary)
print(summary)

'Asked by lawyer Keisha Rice about another song he wrote, Take it Back, which contains the lyrics "plagiarism is hidden", Sheeran confirmed that he had written the words. ', '"Those are my lyrics, yep," he said, adding: "Can I give some context to them?"', "She said that if she wanted any context she would ask for it, and went on to ask him about concert footage recorded in Zurich showing him mixing lyrics from Gaye's song with Thinking Out Loud. ",'The court case comes as the singer prepares to launch a North American stadium tour and release a new album. ', 'This video can not be played', 'WATCH: Ed Sheeran arrives at court for copyright trial', 'Earlier on Tuesday, Sheeran\'s lawyer argued that both songs are distinct from each other and that no artist should be allowed to "monopolise" commonly used musical chord progressions. ','Seeking $100m (£90m) in damages, they allege that Sheeran and his co-writer Amy Wadge "copied and exploited, without authorisation or credit" the Gaye song